In [1]:
pip install faker

Note: you may need to restart the kernel to use updated packages.



[notice] A new release of pip is available: 25.1.1 -> 26.0.1
[notice] To update, run: python.exe -m pip install --upgrade pip


In [3]:
import pandas as pd
import numpy as np
from faker import Faker 
import random
from datetime import datetime,timedelta

In [6]:
fake= Faker()
random.seed(42)
np.random.seed(42)

In [9]:
content_categories= [
    "Hate Speech",
    "Child Safety",
    "Self Harm",
    "Bullying",
    "Animal Cruelty",
    "Violence"
]

#Moderation decision outcomes
outcomes= ["Removed", "Kept", "Escalated"]

#Shift timings(moderation runs 24/7)
shifts= ["Morning", "Afternoon", "Night"]

#Team regions
regions= ["APAC", "EMEA", "AMER"]

#20 fake reviewer IDs
reviewer_ids= [f"REV_{str(i).zfill(3)}" for i in range(1,21)]

In [12]:
records= []

#Generate 6 months of daily data
start_date= datetime(2024,1,1)
end_date= datetime(2024,6,30)
delta= end_date- start_date

for day_offset in range(delta.days):
    current_date = start_date + timedelta(days=day_offset)

    #between 150-300 moderation actions per day
    daily_volume = random.randint(150,300)

    for _ in range(daily_volume):
        reviewer= random.choice(reviewer_ids)
        category= random.choice(content_categories)
        shift= random.choice(shifts)
        region= random.choice(regions)
        
        # stimulate a performance drop in march for root cause analysis later
        if current_date.month==3 and category== "Hate Speech":
            #accuracy drops- more wronf decisions
            correct_decision= random.choices([True, False], weights=[60,40])[0]
        else:
            correct_decision= random.choices([True, False], weights=[88,12])[0]

        #processing time in minutes
        processing_time= round(random.uniform(1.5,15.0),2)

        #SLA breach if processing time>10 mins
        sla_breached= processing_time >10.0

        records.append({
            "date": current_date.strftime("%Y-%m-%d"),
            "reviewer_id": reviewer,
            "content_category": category,
            "shift": shift,
            "region": region,
            "decision_outcome": random.choice(outcomes),
            "correct_decision": correct_decision,
            "processing_time_mins": processing_time,
            "sla_breached": sla_breached
        })
df= pd.DataFrame(records)
print(df.shape)
df.head()

    

(40391, 9)


,date,reviewer_id,content_category,shift,region,decision_outcome,correct_decision,processing_time_mins,sla_breached
0,2024-01-01,REV_020,Hate Speech,Night,APAC,Kept,True,10.97,True
1,2024-01-01,REV_008,Bullying,Night,EMEA,Removed,True,1.59,False
2,2024-01-01,REV_014,Self Harm,Afternoon,APAC,Removed,True,11.81,True
3,2024-01-01,REV_003,Bullying,Morning,EMEA,Removed,True,9.65,False
4,2024-01-01,REV_015,Animal Cruelty,Morning,EMEA,Escalated,True,5.46,False


In [19]:
df.to_csv('C:/Users/vbsr0/Desktop/softwares and notes-20250719T171616Z-1-001/python projects/moderation_dashboard/data/moderation_raw.csv',index= False)
print("Data saved successfully!!")
print(f"Total records: {len(df)}")
print(f"\nDate range:{df["date"].min()} to {df["date"].max()}")
print(f"\nCategory breakdown:\n{df["content_category"].value_counts()}")

Data saved successfully!!
Total records: 40391

Date range:2024-01-01 to 2024-06-29

Category breakdown:
content_category
Child Safety      6834
Hate Speech       6820
Bullying          6756
Self Harm         6741
Animal Cruelty    6648
Violence          6592
Name: count, dtype: int64
